In [1]:
import os
import sys
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import tadpose as tp

import h5py
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import skimage as ski
import yaml

import dask
import dask.distributed
import dask.dataframe as dd

import functools

sys.path.append("frog_analysis/analysis_episodes/")
from features import (
    angle_correlation_episode,
    angle_range_episode,
    frequency_episode,
    locomotion_episode,
)


# @functools.cache
def get_tad(fn):
    return tp.Tadpole.from_sleap(fn)


def read_kpms_results(KPMS_RESULTS_H5):
    KPMS_SYLLABLES = {}
    with h5py.File(KPMS_RESULTS_H5, "r") as hf:
        for k, v in hf.items():
            mov_fn = k[:-12]  # remove suffic ".predictions.slp"
            KPMS_SYLLABLES[mov_fn] = v["syllable"][:]
    return KPMS_SYLLABLES


def kpms_syllables_to_df(SLP_FILES, KPMS_SYLLABLES):
    df_result = []
    for slp_fn in SLP_FILES:
        mov_fn = Path(slp_fn[:-16])
        assert mov_fn.exists()  # Check movie exists
        assert mov_fn.name in KPMS_SYLLABLES  # Check movie was processed

        syllable_seq = KPMS_SYLLABLES[mov_fn.name]

        syllable_seq_lab = ski.measure.label(syllable_seq, background=-1)

        syl_props = ski.measure.regionprops_table(
            syllable_seq_lab[:, None],
            intensity_image=syllable_seq[:, None],
            properties=("area", "label", "mean_intensity", "bbox"),
        )
        tab = pd.DataFrame(syl_props)
        tab.insert(0, "video_fn", str(mov_fn))
        tab["track_idx"] = 0

        df_result.append(tab)

    df_tab = pd.concat(df_result, axis=0, ignore_index=True)
    df_tab.drop(columns=["bbox-1", "bbox-3"], inplace=True)
    df_tab.rename(
        columns={
            "area": "duration",
            "mean_intensity": "syllable",
            "bbox-0": "episode_start",
            "bbox-2": "episode_stop",
            "label": "episode_id",
        },
        inplace=True,
    )
    df_tab["syllable"] = df_tab["syllable"].astype(int)
    df_tab["stage"] = "Juv"
    df_tab["genotype"] = "WT"

    return df_tab


def angle_corr_apply(df_part, cfgs):
    # Apply the function to each row
    res = []
    for _, row in df_part.iterrows():
        if "ANGLE_CORR_FOR" in cfgs:
            for name, nodes_tuple in cfgs["ANGLE_CORR_FOR"].items():
                episode_result = angle_correlation_episode(
                    get_tad(row.video_fn), row, name, nodes_tuple, cfg=cfg
                )
                res.append(episode_result)

    return pd.DataFrame(res)


def angle_range_apply(df_part, cfgs):
    # Apply the function to each row
    res = []
    for _, row in df_part.iterrows():
        if "ANGLE_CORR_FOR" in cfgs:
            for name, nodes_tuple in cfgs["ANGLE_RANGE_FOR"].items():
                episode_result = angle_range_episode(
                    get_tad(row.video_fn), row, name, nodes_tuple, cfg=cfg
                )
                res.append(episode_result)

    return pd.DataFrame(res)


def frequency_apply(df_part, bgrd_sub, cfgs):
    # Apply the function to each row
    res = []
    for _, row in df_part.iterrows():
        if "ANGLE_CORR_FOR" in cfgs:
            for name, nodes_tuple in cfgs["FREQ_FOR"].items():
                episode_result = frequency_episode(
                    get_tad(row.video_fn),
                    row,
                    name,
                    nodes_tuple,
                    bgrd_sub,
                    cfg=cfg,
                )
                res.append(episode_result)

    return pd.DataFrame(res)


def locomotion_apply(df_part, cfgs):
    # Apply the function to each row
    res = []
    for _, row in df_part.iterrows():
        episode_result = locomotion_episode(get_tad(row.video_fn), row, cfg=cfg)
        res.append(episode_result)

    return pd.DataFrame(res)


In [2]:
# the result.h5 in the model directory
KPMS_RESULTS_H5 = "/nfs/scistore08/imagegrp/csommer/projects/sweeney/kpms_2025/wt_kpms/wt_train_pred_v01/kappa=1e7_v01/results.h5"

# Path to the SLEAP .slp output
SLP_PATH = "/nfs/scistore08/imagegrp/csommer/projects/sweeney/kpms_2025/wt_kpms/data/WT"

# Yaml analysis config file
CFG_FN = "/nfs/scistore08/imagegrp/csommer/projects/sweeney/kpms_2025/frog_analysis/MNV1_SETTINGS_final.yaml"

In [3]:
# all WT movies
SLP_FILES = list(map(str, Path(SLP_PATH).glob("*.slp")))
KPMS_SYLLABLES = read_kpms_results(KPMS_RESULTS_H5)

assert len(SLP_FILES) == len(KPMS_SYLLABLES)


In [4]:
DF = kpms_syllables_to_df(SLP_FILES, KPMS_SYLLABLES)
print("Number of Syllable Episodes: (total)", len(DF))

Number of Syllable Episodes: (total) 106722


In [5]:
with open(CFG_FN, "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)

In [ ]:
df = DF[DF.duration > 29]
print("Number of Syllable Episodes:", len(df))
print(
    f"These episodes cover ~{df.duration.sum() / DF.duration.sum():0.2f} % of original durations"
)

Number of Syllable Episodes: 50905
These episodes cover ~0.91 % of original durations


In [7]:
N_PARTITIONS = 60  #
N_WORKERS = 64

from dask.distributed import LocalCluster

cluster = LocalCluster(
    n_workers=N_WORKERS, processes=True
)  # Fully-featured local Dask cluster
client = cluster.get_client()


In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 64,Total memory: 256.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38719,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34021,Total threads: 1
Dashboard: http://127.0.0.1:43209/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:36067,


In [ ]:
###
ddf = dd.from_pandas(df, npartitions=N_PARTITIONS)
###


meta_ac = pd.DataFrame(
    columns=angle_corr_apply(df.iloc[:1], cfgs=cfg["Juv"]).columns.tolist()
)
meta_ar = pd.DataFrame(
    columns=angle_range_apply(df.iloc[:1], cfgs=cfg["Juv"]).columns.tolist()
)
meta_fr = pd.DataFrame(
    columns=frequency_apply(df.iloc[:1], False, cfgs=cfg["Juv"]).columns.tolist()
)


meta_lo = pd.DataFrame(
    columns=locomotion_apply(df.iloc[:1], cfgs=cfg["Juv"]).columns.tolist()
)

angle_corr_ddf = ddf.map_partitions(angle_corr_apply, meta=meta_ac, cfgs=cfg["Juv"])
angle_range_ddf = ddf.map_partitions(angle_range_apply, meta=meta_ar, cfgs=cfg["Juv"])
locomotion_ddf = ddf.map_partitions(locomotion_apply, meta=meta_lo, cfgs=cfg["Juv"])
frequency_ddf = ddf.map_partitions(
    frequency_apply, meta=meta_fr, bgrd_sub=False, cfgs=cfg["Juv"]
)
frequency_BS_ddf = ddf.map_partitions(
    frequency_apply, meta=meta_fr, bgrd_sub=True, cfgs=cfg["Juv"]
)

print("Angle range...")
angle_range_df = angle_range_ddf.compute()
angle_range_df.to_csv("kpms_angle_range_30+.tab", sep="\t")

print("Frequency...")
frequency_df = frequency_ddf.compute()
frequency_df.to_csv("kpms_freuquency_30+.tab", sep="\t")

print("Frequency BS...")
frequency_BS_df = frequency_BS_ddf.compute()
frequency_BS_df.to_csv("kpms_freuquency_BS_30+.tab", sep="\t")

print("Locomotion correlation...")
locomotion_df = locomotion_ddf.compute()
locomotion_df.to_csv("kpms_locomotion_30+.tab", sep="\t")

print("Angle correlation...")
angle_corr_df = angle_corr_ddf.compute()
angle_corr_df.to_csv("kpms_angle_corr_30+.tab", sep="\t")


Angle correlation...


/nfs/scistore08/imagegrp/csommer/.conda/envs/keypoint_moseq/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/nfs/scistore08/imagegrp/csommer/.conda/envs/keypoint_moseq/lib/python3.10/site-packages/numpy/core/_methods.py:152: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/nfs/scistore08/imagegrp/csommer/.conda/envs/keypoint_moseq/lib/python3.10/site-packages/numpy/lib/function_base.py:4657: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
/nfs/scistore08/imagegrp/csommer/.conda/envs/keypoint_moseq/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/nfs/scistore08/imagegrp/csommer/.conda/envs/keypoint_moseq/lib/pyt

In [ ]:
1